# **Data Collection and Transformation**


In [2]:
import pandas as pd
import numpy as np
import wikipedia as wiki
import requests as rq
import bs4 as soup
import io
import re

## **Collecting the Data**
We start by accessing the Paleobiology Database through their [data service API](https://paleobiodb.org/data1.2/). The criteria I am using is based off the Taxonomy of Fossil Occurrences Dataset. Cleaning up the data leaves us with the Classifications, Diet, and First/Last Appearances in the fossil record.

In [3]:
taxa_url = rq.get('https://paleobiodb.org/data1.2/occs/taxa.csv?base_name=Dinosauria&idreso=species&idqual=certain&pres=regular&max_ma=252&min_ma=65&show=class,size,app,ecospace,img').content
taxa = pd.read_csv(io.StringIO(taxa_url.decode('utf-8')))[['taxon_rank', 'taxon_name', 'genus', 'family', 'taxon_size', 'diet', 'firstapp_max_ma', 'lastapp_min_ma']]

taxa = taxa.dropna(subset=['taxon_name']).query('(taxon_rank == \'genus\') or (taxon_rank == \'species\')')
taxa.columns = ['Rank', 'Name', 'Genus', 'Family', 'Taxon Size', 'Diet', 'Max MYA', 'Min MYA']

taxa = taxa.replace(regex=['NO_FAMILY_SPECIFIED'], value='')

taxa['Diet'] = taxa['Diet'].str.capitalize()

taxa.head()

,Rank,Name,Genus,Family,Taxon Size,Diet,Max MYA,Min MYA
10,genus,Ajkaceratops,Ajkaceratops,,2.0,Herbivore,86.3,83.6
11,species,Ajkaceratops kozmai,Ajkaceratops,,1.0,Herbivore,86.3,83.6
12,genus,Turanoceratops,Turanoceratops,,2.0,Herbivore,93.9,89.8
13,species,Turanoceratops tardabilis,Turanoceratops,,1.0,Herbivore,93.9,89.8
14,genus,Zuniceratops,Zuniceratops,,2.0,Herbivore,93.9,89.8


While the dataset above contains a surplus of information, it does not include locations of where the species have been found. To account for this, I am grabbing another dataset that contains all the known fossil occurrences and their respective origin location. Once merged with the taxa dataset, this data will help analyze where certain dinosaurs reside.

In [4]:
# Scraping data from the Paleobiology Database and cleaning up the Null values
occ_url = rq.get('https://paleobiodb.org/data1.2/occs/list.csv?base_name=Dinosauria&taxon_reso=species&idqual=certain&pres=regular&max_ma=252&min_ma=65&show=class,coords,loc,strat,acconly').content
occ = pd.read_csv(io.StringIO(occ_url.decode('utf-8')))[['accepted_name', 'lng', 'lat', 'formation']]

occ.columns = ['Name', 'Longitude', 'Latitude', 'Formation']

occ.head()

,Name,Longitude,Latitude,Formation
0,Chaoyangsaurus youngi,123.966698,42.933300,Tuchengzi
1,Protarchaeopteryx robusta,120.733330,41.799999,Yixian
2,Caudipteryx zoui,120.733330,41.799999,Yixian
3,Gorgosaurus libratus,-111.528732,50.740726,Dinosaur Park
4,Gorgosaurus libratus,-111.549347,50.737015,Dinosaur Park


## **Cleaning the Data**
With the both of these dataframes at our disposal, the next step is to clean it up in a presentable format. Firstly, we want to fix the null values in the Age Columns and add in the corresponding Period and Epochs. To do this, we can sort by year and then choose the Period and Epoch based on what Age the species or genus lived in.

In [5]:
# Looking at the dataframe, we need to clean the columns relating to species lifetime --> Some dinosaurs have NaN as their entries for Early and Late Ages
periods = ['Triassic', 'Jurassic', 'Cretaceous']
epochs = ['Lower', 'Middle', 'Upper']


tri_ages = ['Induan', 'Olenekian', 'Anisian', 'Ladinian', 'Carnian', 'Norian', 'Rhaetian']
jur_ages = ['Hettangian', 'Sinemurian', 'Pliensbachian', 'Toarcian', 'Aalenian', 'Bajocian', 'Bathonian', 'Callovian', 'Oxfordian', 'Kimmeridgian', 'Tithonian']
cre_ages = ['Berriasian', 'Valanginian', 'Hauterivian', 'Barremian', 'Aptian', 'Albian', 'Cenomanian', 'Turonian', 'Coniacian', 'Santonian', 'Campanian', 'Maastrichtian']

low_ep = ['Induan', 'Olenekian', 'Hettangian', 'Sinemurian', 'Pliensbachian', 'Toarcian', 'Berriasian', 'Valanginian', 'Hauterivian', 'Barremian', 'Aptian', 'Albian']
mid_ep = ['Anisian', 'Ladinian', 'Aalenian', 'Bajocian', 'Bathonian', 'Callovian']
upp_ep = ['Carnian', 'Norian', 'Rhaetian', 'Oxfordian', 'Kimmeridgian', 'Tithonian', 'Cenomanian', 'Turonian', 'Coniacian', 'Santonian', 'Campanian', 'Maastrichtian']


ages = [*tri_ages, *jur_ages, *cre_ages]

# Arguements for Age
mya_args = lambda x : [(taxa[x] <= 251.9) & (taxa[x] > 251.2), 
                (taxa[x] <= 251.2) & (taxa[x] > 247.2),
                (taxa[x] <= 247.2) & (taxa[x] > 242),
                (taxa[x] <= 242) & (taxa[x] > 237),
                (taxa[x] <= 237) & (taxa[x] > 227),
                (taxa[x] <= 227) & (taxa[x] > 208.5),
                (taxa[x] <= 208.5) & (taxa[x] > 201.4),
                (taxa[x] <= 201.4) & (taxa[x] > 199.5),
                (taxa[x] <= 199.5) & (taxa[x] > 192.9),
                (taxa[x] <= 192.9) & (taxa[x] > 184.2),
                (taxa[x] <= 184.2) & (taxa[x] > 174.7),
                (taxa[x] <= 174.7) & (taxa[x] > 170.9),
                (taxa[x] <= 170.9) & (taxa[x] > 168.2),
                (taxa[x] <= 168.2) & (taxa[x] > 165.3),
                (taxa[x] <= 165.3) & (taxa[x] > 161.5),
                (taxa[x] <= 161.5) & (taxa[x] > 154.8),
                (taxa[x] <= 154.8) & (taxa[x] > 149.2),
                (taxa[x] <= 149.2) & (taxa[x] > 145),
                (taxa[x] <= 145) & (taxa[x] > 139.8),
                (taxa[x] <= 139.8) & (taxa[x] > 132.6),
                (taxa[x] <= 132.6) & (taxa[x] > 125.77),
                (taxa[x] <= 125.77) & (taxa[x] > 121.4),
                (taxa[x] <= 121.4) & (taxa[x] > 113),
                (taxa[x] <= 113) & (taxa[x] > 100.5),
                (taxa[x] <= 100.5) & (taxa[x] > 93.9),
                (taxa[x] <= 93.9) & (taxa[x] > 89.8),
                (taxa[x] <= 89.8) & (taxa[x] > 86.3),
                (taxa[x] <= 86.3) & (taxa[x] > 83.6),
                (taxa[x] <= 83.6) & (taxa[x] > 72.1),
                (taxa[x] <= 72.1) & (taxa[x] > 66)] 
                
# Arguments for Period and Epoch (will combine into one column later)
pers = lambda x : [(taxa[x].isin(tri_ages)),
                   (taxa[x].isin(jur_ages)),
                   (taxa[x].isin(cre_ages))]

eps = lambda x: [(taxa[x].isin(low_ep)),
                 (taxa[x].isin(mid_ep)),
                 (taxa[x].isin(upp_ep))]


# Adding the Period and Age columns
taxa['Early Age'] = np.select(mya_args('Max MYA'), ages, default=pd.NaT)

# We add 0.01 to accomodate for edge cases where a dinosaur is estimated to have lived at the cusp of two mesozoic ages
taxa['Min MYA'] += 0.01
taxa['Late Age'] = np.select(mya_args('Min MYA'), ages, default=pd.NaT)
taxa['Min MYA'] -= 0.01
taxa['Late Age'] = taxa['Late Age'].fillna(taxa['Early Age'])

taxa['Early Period'] = np.select(eps('Early Age'), epochs, default=pd.NaT) + ' ' + np.select(pers('Early Age'), periods, default=pd.NaT)
taxa['Late Period'] = np.select(eps('Late Age'), epochs, default=pd.NaT) + ' ' + np.select(pers('Late Age'), periods, default=pd.NaT)

# Adding a lifespan column to show how long each species/genus lived
taxa['Lifespan (MYA)'] = taxa['Max MYA'] - taxa['Min MYA']
taxa.head()

,Rank,Name,Genus,Family,Taxon Size,Diet,Max MYA,Min MYA,Early Age,Late Age,Early Period,Late Period,Lifespan (MYA)
10,genus,Ajkaceratops,Ajkaceratops,,2.0,Herbivore,86.3,83.6,Santonian,Santonian,Upper Cretaceous,Upper Cretaceous,2.7
11,species,Ajkaceratops kozmai,Ajkaceratops,,1.0,Herbivore,86.3,83.6,Santonian,Santonian,Upper Cretaceous,Upper Cretaceous,2.7
12,genus,Turanoceratops,Turanoceratops,,2.0,Herbivore,93.9,89.8,Turonian,Turonian,Upper Cretaceous,Upper Cretaceous,4.1
13,species,Turanoceratops tardabilis,Turanoceratops,,1.0,Herbivore,93.9,89.8,Turonian,Turonian,Upper Cretaceous,Upper Cretaceous,4.1
14,genus,Zuniceratops,Zuniceratops,,2.0,Herbivore,93.9,89.8,Turonian,Turonian,Upper Cretaceous,Upper Cretaceous,4.1


Next, we should separate the Genera and Species from each other for more efficient tracking. The dataset was designed to include the Genus in the taxon size count, so we will have to account for that when we separate the Genus.

In [6]:
species = taxa.loc[taxa['Rank'] == 'species'].reset_index().drop(columns=['Rank', 'Taxon Size', 'Family', 'index'])
genus = taxa.loc[taxa['Rank'] == 'genus'].reset_index().drop(columns=['Rank', 'Genus', 'index'])

# Dropping this count by 1 because because the genus in the original dataframe was counted towards the taxon size
genus['Taxon Size'] = genus['Taxon Size'].astype(int) - 1

genus.head()

,Name,Family,Taxon Size,Diet,Max MYA,Min MYA,Early Age,Late Age,Early Period,Late Period,Lifespan (MYA)
0,Ajkaceratops,,1,Herbivore,86.3,83.6,Santonian,Santonian,Upper Cretaceous,Upper Cretaceous,2.7
1,Turanoceratops,,1,Herbivore,93.9,89.8,Turonian,Turonian,Upper Cretaceous,Upper Cretaceous,4.1
2,Zuniceratops,,1,Herbivore,93.9,89.8,Turonian,Turonian,Upper Cretaceous,Upper Cretaceous,4.1
3,Bagaceratops,Protoceratopsidae,1,Herbivore,83.6,66.0,Campanian,Maastrichtian,Upper Cretaceous,Upper Cretaceous,17.6
4,Breviceratops,Protoceratopsidae,1,Herbivore,83.6,72.1,Campanian,Campanian,Upper Cretaceous,Upper Cretaceous,11.5


***Some things to note are that several of the generas in the data set are informally named, named in a way that would be recognized in a web scrape, do not have enough research to assign them to a specific clade, or are trace/egg fossils (which get their own separate genera). Thus, we need to clean these cells up and add some more indentification columns. The following lines acknowledge these discrepancies:***

In [10]:
nondinos = ['Krzyzanowskisaurus', 'Actiosaurus', 'Volgavis', 'Yacoraitichnus', 'Yezosaurus', 'Brasileosaurus', 'Subtiliolithus', 'Fuscinapedis', 'Himeoolithus', 'Protoavis', 'Salfitichnus', 'Nyasasaurus']

# Mislabelled dinosaurs
genus.at[genus.index[genus['Name'] == 'Megalosaurus (Poekilopleuron)'].values[0], 'Name'] = 'Poekilopleuron'
genus.at[genus.index[genus['Name'] == 'Jingia'].values[0], 'Name'] = 'Jingiella'
genus.at[genus.index[genus['Name'] == 'Bellulia'].values[0], 'Name'] = 'Bellulornis'

# take away try clause after completing project
for i in nondinos:
    x = genus.index[genus['Name'] == i].values[0]
    genus = genus.drop(axis=0, index=x)

genus = genus.reset_index()


already changed


In [10]:
# Helper function that finds the corresponding wikipedia page for each dinosaur
def get_webpage(genus):
    # Edge case when looking up Qianlong (Qianlong redirects to the Qianlong Emperor)
    if genus == 'Qianlong':
        print('getting to Qianlong')
        return get_webpage('Qianlong shouhu')
   
   # Web scraping each genus name to figure out classification
    try:
        page = f'https://en.wikipedia.org/wiki/{genus}'
        response = rq.get(page)
        print(response)
      
        biota = soup.BeautifulSoup(response.text, 'html.parser').find('table', {'class': 'infobox biota'}).find_all('tr')
      
    except:
      
      # Covering case where the genus is named after an existing topic
        try:
            page = f'https://en.wikipedia.org/wiki/{genus}_(dinosaur)'
            response = rq.get(page)
      
            biota = soup.BeautifulSoup(response.text, 'html.parser').find('table', {'class': 'infobox biota'}).find_all('tr')
         
      
        except:
            pass
   
    return biota

In [13]:
orders = ['Ornithischia', 'Saurischia']
suborders = ['Neornithischia', 'Thyreophora', 'Sauropodomorpha', 'Theropoda']
infraorders = ['Ornithopoda', 'Ceratopsia', 'Pachycephalosauria', 'Stegosauria', 'Ankylosauria', 'Prosauropoda', 'Sauropoda', 'Tetanurae', 'Coelophysoidea', 'Ceratosauria']
# Coelophysidea, Ceratosauria, Prosauropoda will need to be recategorized manually within the for loop

# Creating a dictionary to use when the dinosaur is noted to be avian
avian = {'Order': 'Saurischia', 'Suborder': 'Theropoda', 'Infraorder': 'Maniraptora', 'Family': '', 'Avialae': True}

def wiki_scrape_genus(genus):
   # Defining the dictionary that will be put into the dataframe
   data = {'Order':'', 'Suborder':'', 'Infraorder':'', 'Family':'', 'Avialae': False}
   
   biota = get_webpage(genus)
   
   # iterating through rows that feature classification terms
   for row in biota:
      biota_data = row.find_all('td')
      
      try:
         bio_class = re.match(r'([A-Z][a-z]*)', biota_data.pop(0).text.strip())[0]
         
         # Only iterating through Clade
         if bio_class in ['Clade', 'Family', 'Genus']:
            bio_name = re.match(r'([^A-Za-z][A-Z][a-z]*)|([A-Z][a-z]*)', biota_data.pop(0).text.strip())[0]
            bio_name = re.sub('†', '', bio_name)
            
            #print(bio_class + " " + bio_name)
            if bio_class in ['Clade', 'Superfamily']:
               if bio_name in orders:
                  #print("SETTING ORDER TO " + bio_name)
                  data['Order'] = bio_name
                  
               elif bio_name in suborders:
                  #print("SETTING SUBORDER TO " + bio_name)
                  data['Suborder'] = bio_name
                  
               elif bio_name in infraorders:
                  #print("SETTING INFRAORDER TO " + bio_name)
                  data['Infraorder'] = bio_name
                     
               elif bio_name in ['Avialae', 'Euornithes']:
                  data = avian
                  
         
            elif bio_class == 'Family':
               #print("SETTING FAMILY TO " + bio_name)
               data['Family'] = bio_name
             
               if bio_name == 'Coelophysidae':
                  data['Infraorder'] = 'Coelophysoidea'
               if bio_name == 'Ceratosauridae':
                  data['Infraorder'] = 'Ceratosauria'                
            
            # Handling edge cases manually where the Infraorder is known, but there is no family      
            elif bio_class == 'Genus':
               
               if bio_name != genus:
                  #print('Checking ' + bio_class)
                  return wiki_scrape_genus(bio_name)             
               
         # In the event that the wikipedia page is not labelled with Dinosauria
         if bio_class == 'Class':
            bio_name = re.match(r'([^A-Za-z][A-Z][a-z]*)|([A-Z][a-z]*)', biota_data.pop(0).text.strip())[0]
            
            if bio_name == 'Aves':
               data = avian       
      
      except:
         pass
      
   return data

print(wiki_scrape_genus('Qianlong'))

getting to Qianlong


UnboundLocalError: cannot access local variable 'biota' where it is not associated with a value

In [9]:
genus['Order'] = ''
genus['Suborder'] = ''
genus['Infraorder'] = ''

genus = genus[['Name', 'Family', 'Infraorder', 'Suborder', 'Order', 'Taxon Size', 'Diet', 'Max MYA', 'Min MYA', 'Lifespan (MYA)', 'Early Age', 'Late Age', 'Early Period', 'Late Period']]

for i, row in genus.iterrows():
    
    try:
        test = genus.iloc[i]['Name']
        wiki_data = wiki_scrape_genus(test)
        for col in wiki_data:
            genus.loc[i, col] = wiki_data[col]
    except:
        if genus.iloc[i]['Family'] != '':
            try:
                test = genus.iloc[i]['Family']
                wiki_data = wiki_scrape_genus(test)
                for col in wiki_data:
                    genus.loc[i, col] = wiki_data[col]
            except:
                pass
            
    
genus.loc[genus['Order'] == 'Saurischia']


getting to Qianlong


IndexError: single positional indexer is out-of-bounds

In [9]:
genus.to_csv('../data-reserve/scraped-genera.csv', index=False)
genus.loc[genus['Order'] == ''].to_csv('../data-reserve/missing-infraorders.csv', index=False)

## Need to make sure all of Sauropodomorpha and Therapoda are covered (Need to account for all infraorders)

In [12]:
#species.loc[species['Genus'] == 'Jingia']
genus.loc[genus['Name'] == 'Avimaia']

,Name,Family,Infraorder,Suborder,Order,Taxon Size,Diet,Max MYA,Min MYA,Lifespan (MYA),Early Age,Late Age,Early Period,Late Period,Avialae
860,Avimaia,Pengornithidae,Maniraptora,Theropoda,Saurischia,1,Carnivore,121.4,113.0,8.4,Aptian,Aptian,Lower Cretaceous,Lower Cretaceous,True
